In [14]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [16]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [17]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [18]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [28]:
ReflectiveDLL_ProcessInjection = spark.sql(
'''
SELECT 
    b.process_path,
    b.process_target_name,
    b.process_target_id,
    b.thread_new_id,
    a.process_id,
    a.process_granted_access
FROM sysmon_events b
JOIN sysmon_events a
ON a.process_guid = b.process_guid
AND a.event_id = 10
AND (a.process_granted_access & 5178) == 5178

WHERE b.event_id = 8
AND NOT b.process_name = "csrss.exe"
'''
).show(10,False)


+---------------------------------------------------------+-------------------+-----------------+-------------+----------+----------------------+
|process_path                                             |process_target_name|process_target_id|thread_new_id|process_id|process_granted_access|
+---------------------------------------------------------+-------------------+-----------------+-------------+----------+----------------------+
|c:\windows\system32\windowspowershell\v1.0\powershell.exe|notepad.exe        |3124             |7940         |5452      |2047999               |
|c:\windows\system32\windowspowershell\v1.0\powershell.exe|notepad.exe        |3124             |7940         |5452      |2047999               |
|c:\windows\system32\windowspowershell\v1.0\powershell.exe|notepad.exe        |7924             |7388         |5452      |2047999               |
|c:\windows\system32\windowspowershell\v1.0\powershell.exe|notepad.exe        |7924             |7388         |5452      |20